In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import seaborn as sns
import datajoint as dj
from funconnect import connectomics as cncs, ta3, nda
from funconnect.connectomics import *
from funconnect.morphology import *
import numpy as np
from igraph import *
import pandas as pd

In [ ]:
df0 = pd.DataFrame((ta3.Segment() * MeshSurfaceArea - ta3.NeuriteSkeleton).proj(
    'surface_area', rank='RAND()', type='"no skeleton"').fetch(limit=10000, order_by='rank'))
df1 = pd.DataFrame((ta3.Segment() * MeshSurfaceArea & ta3.NeuriteSkeleton).proj(
    'surface_area', rank='RAND()', type='"skeleton available"').fetch(limit=10000, order_by='rank'))
len(df0), len(df1)

In [ ]:
df = pd.concat([df0, df1])

In [ ]:
with sns.axes_style('ticks'):
    g = sns.FacetGrid(df, hue='type')
#     g.map(sns.distplot, 'surface_area', bins=np.linspace(0, 100, 50), kde=False, norm_hist=False)
    g.map(sns.kdeplot, 'surface_area', shade=True, lw=1, clip=(0, 100), bw=2)
g.fig.set_dpi(200)
g.ax.set_yscale('log')
g.add_legend(title='selection')
g.ax.set_ylabel('#segments')
g.ax.set_xlabel(r'mesh surface area $\mu m^2$')
g.fig.savefig('figures/surface_neurite_vs_not.png')
g.fig.savefig('figures/surface_neurite_vs_not.pdf')


In [ ]:
dj.U('segmentation').aggr((ta3.Segment() * MeshSurfaceArea - ta3.NeuriteSkeleton), mean_surface='AVG(surface_area)')

In [ ]:
dj.U('segmentation').aggr((ta3.Segment() * MeshSurfaceArea & ta3.NeuriteSkeleton), mean_surface='AVG(surface_area)')